In [ ]:
# Iterative RAFFLE structure search
from ase.io import read
from raffle.generator import raffle_generator
from mace.calculators import mace_mp
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import AutoMinorLocator

In [ ]:
# Set the font family and size to use for Matplotlib figures
plt.rc('text', usetex=True)
plt.rc('font', family='Computer Modern')
plt.rcParams.update({
    "figure.facecolor":  (1.0, 1.0, 1.0, 1.0),  # red   with alpha = 30%
    "axes.facecolor":    (1.0, 1.0, 1.0, 1.0),  # green with alpha = 50%
    "savefig.facecolor": (1.0, 1.0, 1.0, 0.0),  # blue  with alpha = 20%
})

In [ ]:
generator = raffle_generator()
calc = mace_mp(model="medium", dispersion=False, default_dtype="float32", device='cpu')

In [ ]:
aa_stack = read("POSCAR_AA_stack")
ab_stack = read("POSCAR_AB_stack")
abc_stack = read("POSCAR_ABC_stack")
aabbcc_stack = read("POSCAR_AABBCC_stack")
aba_stack = read("POSCAR_ABA_stack")
abab_stack = read("POSCAR_ABAB_stack")
lonsdaleite = read("POSCAR_lonsdaleite")
diamond = read("POSCAR_diamond")

# 0 = mp-568806
# 1 = mp-169 = 0.001 OR mp-3347313 = 0.000
# 2 = mp-2516584 = 0.002
# 3 = mp-606949 = 0.006
# 4 = mp-569416 = 0.002
# 5 = mp-47 = 0.139
# 6 = mp-66 = 0.112

In [ ]:
aa_stack.calc = calc
ab_stack.calc = calc
abc_stack.calc = calc
aabbcc_stack.calc = calc
aba_stack.calc = calc
abab_stack.calc = calc
lonsdaleite.calc = calc
diamond.calc = calc

In [ ]:
# # change aa_stack to same height as ab_stack
# # this is to confirm that the differences in descriptor are not a result of different interlayer distances
# ab_height = ab_stack.get_cell()[2, 2]
# cell = aa_stack.get_cell()
# aa_height = cell[2, 2]
# cell[2, 2] = ab_height
# aa_stack.set_cell(cell, scale_atoms=True)
# aa_stack.calc = calc

In [ ]:
aa_descriptor = generator.distributions.generate_fingerprint(aa_stack)
ab_descriptor = generator.distributions.generate_fingerprint(ab_stack)
abc_descriptor = generator.distributions.generate_fingerprint(abc_stack)
aba_descriptor = generator.distributions.generate_fingerprint(aba_stack)
abab_descriptor = generator.distributions.generate_fingerprint(abab_stack)
aabbcc_descriptor = generator.distributions.generate_fingerprint(aabbcc_stack)
lonsdaleite_descriptor = generator.distributions.generate_fingerprint(lonsdaleite)
diamond_descriptor = generator.distributions.generate_fingerprint(diamond)

In [ ]:
# Create a figure with 3 subplots side by side
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot for each n-body function (2-body, 3-body, 4-body)
for j in range(3):
    # Calculate x-axis values
    x = np.arange(generator.distributions.cutoff_min[j],
                generator.distributions.cutoff_max[j] + generator.distributions.width[j],
                generator.distributions.width[j])

    # Plot on the respective subplot
    axes[j].plot(x, aa_descriptor[j], label='AA stack')
    axes[j].plot(x, ab_descriptor[j], label='AB stack')
    axes[j].plot(x, abc_descriptor[j], label='ABC stack')
    # axes[j].plot(x, aabbcc_descriptor[j], label='AABBCC stack')
    # axes[j].plot(x, aba_descriptor[j], label='ABA stack')
    # axes[j].plot(x, abab_descriptor[j], label='ABAB stack')
    # axes[j].plot(x, lonsdaleite_descriptor[j], label='Lonsdaleite')
    # axes[j].plot(x, diamond_descriptor[j], label='Diamond')

    # change line colours to be more distinct
    axes[j].lines[0].set_color('black')
    axes[j].lines[1].set_color('red')
    axes[j].lines[2].set_color('dodgerblue')
    # axes[j].set_title(f'{j+2}-body fingerprint')
    axes[j].legend()

axes[0].set_ylabel('Distribution function (arb. units)', fontsize=20)
axes[0].set_xlabel('Bond length (Å)', fontsize=20)
axes[1].set_xlabel('3-body angle (radians)', fontsize=20)
axes[2].set_xlabel('4-body angle (radians)', fontsize=20)
axes[0].set_xlim(0, 6)
axes[1].set_xlim(0, np.pi)
axes[2].set_xlim(0, np.pi)

axes[0].set_ylim(0, None)
axes[1].set_ylim(0, 0.2)
axes[2].set_ylim(0, 0.1)

# set the legend font size
for ax in axes:
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(16)
    ax.legend(fontsize=16)

# reduce number of ticks to five at most
axes[0].xaxis.set_major_locator(plt.MaxNLocator(3))
axes[0].yaxis.set_major_locator(plt.MaxNLocator(3))

# set x ticks to 0, 1, 2, 3
axes[1].set_xticks([0, 1, 2, 3])
axes[2].set_xticks([0, 1, 2, 3])
axes[1].set_yticks([0, 0.1, 0.2])
axes[2].set_yticks([0, 0.05, 0.1])

# have the ticks point intwards and on both sides
for ax in axes:
    ax.tick_params(axis='both', which='major', direction='in', length=10, width=1)
    ax.tick_params(axis='both', which='minor', direction='in', length=5, width=1)
    ax.tick_params(axis='x', which='major', length=10, bottom=True, top=True)
    ax.tick_params(axis='y', which='major', length=10, left=True, right=True)

# add minor ticks to all axes
for ax in axes:
    ax.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))

plt.tight_layout()
plt.show()

# save as a pdf
fig.savefig("fingerprints.pdf", bbox_inches='tight', dpi=300)
